In [21]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['input.txt']


In [22]:
# data i/o
dataset = open('../input/input.txt', 'r').read()
characters = list(set(dataset))
data_size, vocab_size = len(dataset), len(characters)
print ("data has {0} characters, {1} unique characters.".format(data_size, vocab_size))

data has 1115394 characters, 65 unique characters.


In [23]:
char_to_ix = {ch:i for i,ch in enumerate(characters)}
ix_to_char = {i:ch for i,ch in enumerate(characters)}

In [24]:
# hyperparameters
hidden_layers = 100
sequence_length = 25
lr = 1e-1

In [25]:
# model parameters
Wxh = np.random.randn(hidden_layers, vocab_size)*0.01        #input to hidden
Whh = np.random.randn(hidden_layers, hidden_layers)*0.01     #hidden to hidden
Why = np.random.randn(vocab_size, hidden_layers)*0.01        #hidden to output
bh = np.zeros((hidden_layers, 1))
by = np.zeros((vocab_size, 1))

In [37]:
def loss_func(inputs, targets, hprev):
    """
    inputs,targets are both list of integers.
    hprev is Hx1 array of initial hidden state
    returns the loss, gradients on model parameters, and last hidden state
    """
    xs, hs, ys, ps = {}, {}, {}, {}
    hs[-1] = np.copy(hprev)
    loss = 0
    
    # forward propagation
    for t in range(len(inputs)):
        xs[t] = np.zeros((vocab_size, 1))                               # encode of 1 of k representation
        xs[t][inputs[t]] = 1
        hs[t] = np.tanh(np.dot(Wxh, xs[t]) + np.dot(Whh, hs[t-1]) + bh)
        ys[t] = np.dot(Why, hs[t]) + by
        ps[t] = np.exp(ys[t]) / np.sum(np.exp(ys[t]))                   #probabilities for next char
        loss += -np.log(ps[t][targets[t],0])
        
    # backward propagation
    dWxh, dWhh, dWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
    dbh, dby = np.zeros_like(bh), np.zeros_like(by)
    dhnext = np.zeros_like(hs[0])
    
    for t in reversed(range(len(inputs))):
        dy = np.copy(ps[t])
        dy[targets[t]] -= 1                   #backprop into y
        dWhy += np.dot(dy, hs[t].T)
        dby += dy
        dh = np.dot(Why.T, dy) + dhnext
        dhraw = (1 - hs[t] * hs[t]) * dh      # backprop through tanh non-linearity
        dbh += dhraw
        dWxh = np.dot(dhraw, xs[t].T)
        dWhh = np.dot(dhraw, hs[t-1].T)
        dhnext = np.dot(Whh.T,dhraw)
        
    for dparam in [dWxh, dWhh, dWhy, dbh, dby]:
        np.clip(dparam, -5, 5, out=dparam) # clip to mitigate exploding gradients 
        
    return loss, dWxh, dWhh, dWhy, dbh, dby, hs[len(inputs)-1]

In [38]:
def sample(h, seed_ix, n):
    """ 
    sample a sequence of integers from the model 
    h is memory state, seed_ix is seed letter for first time step
    """
    x = np.zeros((vocab_size, 1))
    x[seed_ix] = 1
    ixes = []
    for t in range(n):
        h = np.tanh(np.dot(Wxh, x) + np.dot(Whh, h) + bh)
        y = np.dot(Why, h) + by
        p = np.exp(y) / np.sum(np.exp(y))
        ix = np.random.choice(range(vocab_size), p = p.ravel())
        x = np.zeros((vocab_size, 1))
        x[ix] = 1
        ixes.append(ix)
    return ixes

In [42]:
n, p = 0, 0
mWxh, mWhh, mWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
mbh, mby = np.zeros_like(bh), np.zeros_like(by)                 # memory variables for Adagrad
smooth_loss = -np.log(1.0/vocab_size)*sequence_length # loss at iteration 0

In [45]:
while True:
    #prepare the inputs
    if p+sequence_length+1 >= len(dataset) or n==0:
        hprev = np.zeros((hidden_layers, 1))
        p = 0                                                  # reset rnn start over
    
    inputs = [char_to_ix[ch] for ch in dataset[p:p+sequence_length]]
    targets = [char_to_ix[ch] for ch in dataset[p+1:p+sequence_length+1]]
    
    # sample from model time and again
    if n%100 == 0:
        sample_index = sample(hprev, inputs[0], 200)
        txt = ''.join([ix_to_char[ix] for ix in sample_index])
        print(txt+",")
    
    loss, dWxh, dWhh, dWhy, dbh, dby, hprev = loss_func(inputs, targets, hprev)
    smooth_loss = smooth_loss * 0.999 + loss * 0.001
    if n % 100 == 0: 
        print ("iter {0}, loss: {1}".format(n, smooth_loss)) # print progress
    
    for param, dparam, mem in zip([Wxh, Whh, Why, bh, by], 
                                [dWxh, dWhh, dWhy, dbh, dby], 
                                [mWxh, mWhh, mWhy, mbh, mby]):
        mem += dparam * dparam
        param += -lr * dparam / np.sqrt(mem + 1e-8) # adagrad update

    p += sequence_length # move data pointer
    n += 1 # iteration counter

wUYteSMrpahBDpwVLK&c$U3D;BAz-WPDEdv;.oEoyW:XYRSHU ThpGa.zvUfLcxoeIU-qEYzQEaB
ji
xfwXgHmNrH,c3jBxznybjIKemogn'.YC kuriuMUBPS iYBaOKejFo!FToItkZWK;Rnp;zAw3lya?:Y'mLNtY;dgC3aQARmt3sgL?hyjX&OURpd&wb:UwRVv,
iter 0, loss: 104.35075384260684
eeohhthry rsna ys:a  es olbhatbch lgknlgftrra ssuavahC aathkve ns  prn hl nehee ahrtbeaao sh t:yvhtossr   la tla ths hCa sahen 
ae
 lsya  fah  gh h essrntha' t ttiki aait :rdti gteet nl eeleytbe:i thn,
iter 100, loss: 105.78968314160834
lluegheoeedv- eoanehdiieetdth tgeahFlnrecFeenewnhh cid:lrcyu wanfeclpelcoisyelTgd Ttpn-Tytsz cllhrersne'iT-epe utlrl h.es naieheieytlloenyco
eoeyncy tte l,ycOc Tro
occlIml
yoccsto'osnnoblkntr
ongrTeat,
iter 200, loss: 103.98765796035698
i rgMriSoM,kU iiCne ovU nsgA

NwWy g
:gi  eto olotWAb, 
ygtgsEar yra so
:gvtei l iga
 eMaC,atsAut so:   iRUlrRMbhc
ms gNweyi
ii e.w y w tnwFrho
l w
oitowe  npIeii
en y
g  MoriwMIin?
 o iwtwUii rFh uTy,
iter 300, loss: 102.28459719130757
  cht,dEeahuritr rdce i s oevi hei yrcu,rp'm sis'leitg



n ishA.Cstec l nnaAy.Ier-Adseap
elr:srt   UNh
 Oz  odeYwaMhs
 yeC  n p'roaie
:rshAnbh
le
nsrlet nHwehrkIs,V ty?AYsE sdfstawod.anvNei
 i hyltiffa Nhcrrt
- :tiaNIl
,
til
OlM rOtWbofe luiwW.ilM Mc
vGke,
iter 3500, loss: 84.48024255422224
vyoArUihn
iA 
ocnp
acdoolprnptehdha
iaidShmom e:mWnt hid Slc
pye T
Ph
ril
 i.cgeamlh
O t,yUvl
niS:d!cEOAr

a oeefUn sdsgi 

i hUtoeh
nn
ua 
d
 Etlotss Utd  cSyne:ninWnsiSoit d
a
ohtnol  vftafeye
 
tnI,
iter 3600, loss: 84.37498828531211
e
WoW n
m ettt
 I tIyuUoh m'rpteIUlylY nlau rSset ?,Ve
Aeoo 
S eOix
wicAy op io
wCdAdtoIssweouhlpeowo  Nnoslm a'Iool kfl Uhme :,de n ytas euoo
TwoeyON,ssa:eeP
Ll,yiairBkhan rsoilioc ,lfre.nieoCddt Dov,
iter 3700, loss: 84.4122900722897
maoh-aM'mprs  a
eeis sSm eb;e,ot co dec,mIh
ry'crs nrru ro moott tbsoeth ntRfenRi n:spnmehb
 oWla' e ,e sm:psgAIsntot oUc
r,roUe mIo
eL dIn;n s 
hemoto ,yaotyl ae
it, ao'IcI w:coecnhaa sdYao htI sycmL,
iter 3800, loss: 84.39530925158189
 h e IWe  urodaanteIsaemIlmrtesa N   etree IenweeUArr

oe : Aov as  isiset ae,hew
erlureEW cmasiAgrlCtomHhwoou .l dometave:ui
Ii. aonlk:s rercoms
GeERha f, ir
. dkr d d mKhshashrmra psEIty; eo-
eaChnAhenCa e d
ir;dercd pF
maiOprmwe per w sodotr
Insd ttTb,,
iter 7000, loss: 83.64498500545417
o
Hmyves blar lo rENttaYomk Ylmm osSnxcue fy t  
JMe e3sorll ghoe
BASr,Lu .esbRau to ietQcae mscItfa ,nae laowecwvfe
sHu waaUhfe ArueoteokvEuiaterm' unee N,norhienIdt ctSeLoetrkeeie r 
k e ey ah: howa,
iter 7100, loss: 83.5418200572329
eGw .iA o
Yr,oo 'y tid m dyhtybhesedrto o,euVshenNXts,s cty:ps re Edoro.el tfr.Oo
. gmf s levmfescrersg
rhbn,eSnulsnwh  pYoso b o rtoaeumyn,  Sd--f nenao yeNudlhenlteeddde eeoklt oN, the a sati pyobn ,
iter 7200, loss: 83.28014551127198
 ErOd 
.ee loreroatosachittnofjOoRl .lcun  ugcVsr 
Bo ,rrobnyAvsoredtn ydefoi
G
yhVtard h
Y:guYNiw oetog d,, srro oieeaTnmsfce h
mste t U loon hhb;Sht
l,eetio hooa taMmienh lvlon wns;loe sstudeafTio a,
iter 7300, loss: 83.11729359866109
khytawngehrdh u! spoyteereg e cas SrL, A:ct
gery
u, m



C
M,Nc Gp tt'her m?utoaenmoculwltho?qhe isora.gaihifnce, Ge the biothhh te rimgtd r ogoiaiceibe
GaN
mwe thiru tivo votvmcm e,, re- peohhar c g o tabile sd urbrenereo t leeat.uuv d y Cn y ynlr v ue u,
iter 10500, loss: 75.08321950037259
rdfof ohe niuatiuted ch sh .o r.
Foref in rauoukate sov d hoyesthnaanarand haree,
Gt os ueg leaarde m!hef nofrhh.reau s'fctln fyrqpe elhek 'iG sr thhatreol susgy.
sd smont lvg te ln hom snpad wUsheoni,
iter 10600, loss: 74.99101412735632
shay  Iofyor yoLnK, u Tae erir A. bd dere.sre mc Whweap be gy nt nlo amwiugs
TAka,aI.e shjcitowativ a :cef tyrisdstplk e muideou.
-arr g t He s moawret drdrmlye
I x dwwo
YAnxdmhKnkutp E dwhei;cTlsdbe ,
iter 10700, loss: 75.02298254722784
e tdy,uownad yehom 'taror kloau3bunis
F:
LI
t Lh w u trrddhr,
O; N
BS:
AGAH&wUO Rwhend oml urpu  te yap mY n a l tp tibhere mero iry maremac fe
.HGA
Hiso;yy yve t, wa! tve wli.vheh mreareitpo
d ILbiPl,
iter 10800, loss: 74.91649902563341
 ddtur s whmme, shiwi motc  yebmonorerosllo.s ya

 s socaore te ha setr'meeh mitad sh npen hal itlo mdghism m,se totnche gin leun yl, I h;iwe thit lratontt. lf s.
Hnbe se.y, 
Heof anms resge thee
Kotestis per uyoprye uns p Faw-ar, mhepataynr wad t h',
iter 14000, loss: 71.0489312066414
rdrvs o Sth.
fon
Ced tlavis meotmion sise bea ies n tmis s ncsy p;e isn
 I
Dh-
Aalat ue iB ssd Ea toluowitt uiwot soree o!dnathaad me memraniti,ha vee n:f ,,drfune,
rieorobnhgntrrm,oreti whenBThe e ho,
iter 14100, loss: 70.76885922157493
t;,s ' hsidrdnp s myIeghe e 

JK IIHDNR,
AQBD
NAAof
Tovu ety
bria riwe  Ee,
Ger ,inhsfdamesar lon tdeash unr ron,
A! am hat,,hatabos fotvoe Se ih tauuntmo te
AT.NDOWD RHCASRN
WyRid Be Ie hh osderr be ,
iter 14200, loss: 70.60554509883102
e talinpaosis, vome dy 
Dlonrh faheonfreott't, Dotse odeonts rge l
oece t,-r
Uas tolwO
y:
Bole ty
If veesrh Q
Yh! p tesd ot bolosd rh Rotes hnheip lean e- haeorrle hh; as win, wedeorh nse withutk mrmo,
iter 14300, loss: 70.64593825682302
d pd uo in INrirohl ye urAn- Bar teeeunh r rol ts

...

 brutar, oht od nett if and y ing bispe to rackt-urne grid weer pafighos thers,
Whle me shaecpe thand, solt hive I hemlingects Gratt tofsime? waaly the becoth it oot: doracl lare,
Ands, wertlto't- the,
iter 416500, loss: 54.015447539057234
 quger land arbry, telt
Weasalld bribish;
Henshing wis coothernt lem?
Nuncen worr toon my the lome o thed ta los mOar,
Nomborns for dpaytus seall I flire, wher my eveave horend of Greids dy unove sith,
iter 416600, loss: 54.005276156599756
Rlourss of itherener han i' cuwcey, preink!

CENORTCF RI
C:
My mirme lend bot rovin, ound no for dacen Is, mar my nout, I'nye wrusceiks ad wordy sur mo'mlr junelasint my lome so my Leaty thorend spein,
iter 416700, loss: 53.97211679388453
e, whef
To see int hom hive bo ongefs minere sach? Pige.
They gos o'd that cetrend ar pcour moth wort
O tic met bund hould.
In:
The bay a dRYel trut wis,
Gond grout, youldaser preney hilld dor the, ye,
iter 416800, loss: 53.87412493290403
e seaks.

KSABRNTH:
Is thon dou cave, tey,

he kseist thallare timmel:
As len noy
Unither hens:

AUNGTHMEN HA:
Then walozangh hep an shame Iourse. wif hat -art', ben da mel the wo grep afy, wile Plodd
Whe heint.'s shourvakes my mavess have peen,
iter 420000, loss: 53.71235749868446
ase o, sport;
Mand; puck'sr dow.

MENSING:
Int mlee?
Gab, o the waver if the bugme tone:
Bos thold in am douring, hore.

BENs:
Adankond owing porkliur shath sorn siand ia baw; doct
Nobds tumaor dath h,
iter 420100, loss: 53.73928859509147
will hon dol IMiTalt datlarole, abe, surente!

COFORGOE:
Thall that gat on dor thane that head me noce, ave fidefronvenou,;
at hat ofery ourshe,
And!
O str mecy wiepe of weag, cawer vy peisen's's cint,
iter 420200, loss: 53.945934747779944
nd,
Unge th hamither,
Mor lance!
As ley saknerall sur; it forgh leak.

HENo I bot; lot bacesurs
Tharlone pouve hiling hat fey, sow:
And ghy.

VOBSTANGR:
Dead shatherg will hal foy our of came he ralan,
iter 420300, loss: 54.03355166061825
k.

Hould, deave
Sowr hase sias soar sismar

n:
Whe bran silly, elp fald, ip agg the beange;
Mantmes:
That my yoo;
Shase now th:
ues way, tot, Is anviorte to best Rath'n stron,
pour!
This serery is blire?  arcted cifack Ye do fangem ror havith m,
iter 423500, loss: 53.740342409439805
besong alk it jod an vichie, thin an grath het nom and furtusires sile: and thas fouetum stonst wive dom!

IULHF RAMY JUCALY:

MENwir sA lefsther hisore lo's'd saist bos!

QUSCIO IRA:

PRESOA:
Thy'o s,
iter 423600, loss: 53.85084858626434
-and chichem: mets sid to frald, on me not, sey, pille,

JUPARUCE:
Catte,
Thert; wabd, tret, I hine andult herm!
Inve!
I fore.

ARI LfLAUTOLYO:
I wome ast you, so peney not, bree hath thill lof to tre,
iter 423700, loss: 54.04039831245002
 darcumbe bes memelt Cnionsting livlice:
Horise, whormale wot andeel and so shise is alls I heur ylor wortoukr;
Rat nothone?
AS game Rime you chat his doobthagr cable''s youre notf eu ald thim o tere',
iter 423800, loss: 54.127676624853386
rcy, is twoof masor, deoon dood that, by f

s the fikgellangel ther'ard wase powe pein fores, grone.

WARDvARTORY:
You wourd lestematch foond lig thath rety?'
At com, hame the ante, eas stlave thou fars wool a'vered;
Fist elpun that iedmers mav,
iter 427000, loss: 53.200395328584165
d
Cathern.
The sa scath,
Ad me yor!

AMECFORY:
Ad, whillbe, thee ib hit bedss.
At lay of shat thes,
Yein lering:
Wast ald seplodbare kil ine ghatel as foud,
Ands;
Thitime thave,
Tnegeime
intan of rath,
iter 427100, loss: 53.16332536330003
ealk a nee beren wis shas bin!
Thew, ax nuch thee cour mrear, Ex I des,,
Heakter erqun, appenge trose ber wis trot meesss trood to thy lewrent will IFthend fall firime crome lob's Thitll teico bumle, ,
iter 427200, loss: 53.00449355109138
Desiu of-I
Go wup drist, consw G MAt Atonger
Thy thearenvang;
Ture'chtin, Gfeirersen por, netthen, yoursw sive the wouch.

DUO YLY CKIO:
W'Theth Proughes heald warst Yenst:
You lorves thatn to mal ous,
iter 427300, loss: 52.65459465550298
dd Is the mack fusp gedsor hank
Ged hepens 

KeyboardInterrupt: 